In [1]:
# 来自KIMI官方文档: https://platform.moonshot.cn/docs/api/chat
# 使用 OpenAI SDK 时运行文档中的代码时，需要保证 Python 版本至少为 3.7.1，Node.js 版本至少为 18，OpenAI SDK 版本不低于 1.0.0。
# python -c 'import openai; print("version =",openai.__version__)'
# version = 1.28.0

In [20]:
import os
from openai import OpenAI
import utils
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file

MOONSHOT_API_KEY = os.environ['MOONSHOT_API_KEY']
print (MOONSHOT_API_KEY)
LLM_MODEL = "moonshot-v1-8k"

 
client = OpenAI(
    api_key = MOONSHOT_API_KEY,
    base_url = "https://api.moonshot.cn/v1",
)

def get_completion_from_messages(messages, model=LLM_MODEL, temperature=0.6, max_tokens=2800):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    ### token_dict = {'prompt_tokens': response['usage']['prompt_tokens']}
    return response.choices[0].message.content


In [16]:
messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
]
response = get_completion_from_messages(messages)
print(response)

你好，李雷！1+1等于2。很高兴认识你。如果你有任何问题或需要帮助，请随时告诉我。
ChatCompletionMessage(content='你好，李雷！1+1等于2。很高兴认识你。如果你有任何问题或需要帮助，请随时告诉我。', role='assistant', function_call=None, tool_calls=None)


In [28]:
history = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
]
def chat(query, history):
    print(history)
    history.append({
        "role": "user", 
        "content": query
    })
    result = get_completion_from_messages(history)
    
    history.append({
        "role": "assistant",
        "content": result
    })
    return result

print(chat("地球的自转周期是多少？", history))
print(chat("月球呢？", history))




[{'role': 'system', 'content': '你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。'}]
地球的自转周期，即地球自身旋转一圈所需的时间，通常是24小时，也就是一天。但这个周期是基于太阳日来定义的，它指的是太阳从地球上某一点正午时刻再次经过该点所需的时间。实际上，地球相对于固定恒星的自转周期略短于24小时，这个周期称为恒星日，大约是23小时56分钟4秒。这个时间差异是由于地球在自转的同时还在绕太阳公转，所以太阳日比恒星日稍长一些。
[{'role': 'system', 'content': '你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。'}, {'role': 'user', 'content': '地球的自转周期是多少？'}, {'role': 'assistant', 'content': '地球的自转周期，即地球自身旋转一圈所需的时间，通常是24小时，也就是一天。但这个周期是基于太阳日来定义的，它指的是太阳从地球上某一点正午时刻再次经过该点所需的时间。实际上，地球相对于固定恒星的自转周期略短于24小时，这个周期称为恒星日，大约是23小时56分钟4秒。这个时间差异是由于地球在自转的同时还在绕太阳公转，所以太阳日比恒星日稍长一些。'}]
月球的自转周期与它的公转周期是同步的，这意味着月球在绕地球公转一圈的同时完成一次自转。这个周期称为一个朔望月，大约是27.3天。因此，月球总是将同一侧面朝向地球，这种现象称为潮汐锁定。

具体来说，月球的自转周期大约是27天7小时43分11.5秒。但是，由于地球和月球之间的相对位置以及月球轨道的椭圆形状，我们从地球上看到的月球表面会有所变化，这种现象称为天平动。通过天平动，我们可以从地球上额外看到月球表面的大约59%。


In [29]:
#https://platform.moonshot.cn/docs/api/tool-use
#Tool Use 或者 Function Calling 是 Kimi 大模型的一个重要功能，在调用 API 使用模型服务时，您可以在 Messages 中描述工具或函数，并让 Kimi 大模型智能地选择输出一个包含调用一个或多个函数所需的参数的 JSON 对象，实现让 Kimi 大模型链接使用外部工具的目的。

from openai import OpenAI
 
client = OpenAI(
    api_key = "$MOONSHOT_API_KEY",
    base_url = "https://api.moonshot.cn/v1",
)
 
completion = client.chat.completions.create(
    model = "moonshot-v1-8k",
    messages = [
        {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
        {"role": "user", "content": "编程判断 3214567 是否是素数。"}
    ],
    tools = [{
        "type": "function",
        "function": {
            "name": "CodeRunner",
            "description": "代码执行器，支持运行 python 和 javascript 代码",
            "parameters": {
                "properties": {
                    "language": {
                        "type": "string",
                        "enum": ["python", "javascript"]
                    },
                    "code": {
                        "type": "string",
                        "description": "代码写在这里"
                    }
                },
            "type": "object"
            }
        }
    }],
    temperature = 0.3,
)
 
print(completion.choices[0].message)

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid Authentication', 'type': 'invalid_authentication_error'}}